In [33]:
import numpy as np
import os
import pandas as pd
import cv2
import mahotas as mt
import skimage.feature as ftr
import skimage.io
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [34]:
def extract_features(roi):
    textures = mt.features.haralick(roi)
    ht_mean = textures.mean(axis=0)
    return ht_mean

In [35]:
def findGreatesContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)
    while (i < total_contours ):
        area = cv2.contourArea(contours[i])
        if(area > largest_area):
            largest_area = area
            largest_contour_index = i
        i+=1
    return largest_area, largest_contour_index

In [36]:
def getImageFeatures(path):
    
    image = cv2.imread(path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    bilateralFilter = cv2.bilateralFilter(gray, 9, 100, 100)

    thresh = cv2.threshold(bilateralFilter, 150, 200, cv2.THRESH_BINARY)[1]

    thresh = cv2.erode(thresh, None, iterations=2)

    thresh = cv2.dilate(thresh, None, iterations=2)

    _, contours, hierarchy =  cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0 :
        largest_area, largest_contour_index = findGreatesContour(contours)
        x, y, w, h = cv2.boundingRect(contours[largest_contour_index])
        roi = image[y:y+h, x:x+w]
    else:
        roi = image
        
    #exibe retangulo do tumor
    x, y, w, h = cv2.boundingRect(contours[largest_contour_index])
    roi = image[y:y+h, x:x+w]
    cv2.rectangle(image,(x,y),( x + w, y + h ),(0,255,0),2) 
    cv2.imshow('area',image)
    cv2.waitKey(0)
    grayROI = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    features = extract_features(grayROI)

    return features

## Obtendo Feature da Imagem

In [85]:
#filename = './Testes/1no.jpeg'
#filename = './Testes/2no.jpeg'
#filename = './Testes/1y.jpeg'
#filename = './Testes/2y.jpg'
#filename = './Testes/3no.jpg'
#filename = './Testes/3y.jpg'
#filename = './Testes/4y.jpg'
#filename = './Testes/5y.jpg'

filename = './img/Testes/3no.jpg'
df = pd.DataFrame()

In [86]:
features = getImageFeatures(filename)

In [87]:
row = np.array(features)

In [88]:
df = df.append(pd.Series(row), ignore_index=True)

In [89]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.050655,1612.659091,0.347786,1230.954678,0.232727,357.132056,3311.15962,5.903896,7.145022,0.000229,5.390063,-0.609467,0.999004


## Carregando Modelos Standard e Svm salvos

In [90]:
x = df.iloc[:, 0:12]

pickle_in = open("data_Standard_Model.sav","rb")
sc = pickle.load(pickle_in)

x_validate = sc.transform(x)

In [91]:
pkl_filename = "./optimised_model.sav"
pickle_in = open(pkl_filename,"rb")
svm = pickle.load(pickle_in)

In [92]:
predict = svm.predict(x_validate)

In [93]:
predict

array([0.])